In [1]:
import torch
import numpy as np

In [2]:
from caption_decoder import TextDecoder
text_decoder = TextDecoder.from_pretrained('/mnt/bn/us-aigc-temp/henry/unilatent_weights/gpt/', device_map=None, low_cpu_mem_usage=False)
# slightly hacky -- cannot save wte weights since they are shared with lm_head, so we copy them back here
text_decoder.transformer.transformer.wte.weight = text_decoder.transformer.lm_head.weight
text_decoder.encode_prefix = torch.nn.Linear(1024, 768)

from transformers import GPT2Tokenizer
text_tokenizer = GPT2Tokenizer.from_pretrained('/mnt/bn/us-aigc-temp/henry/unilatent_weights/gpt_tokenizer/')
text_tokenizer.add_special_tokens({'pad_token': text_tokenizer.eos_token})

RuntimeError: Error(s) in loading state_dict for TextDecoder:
	size mismatch for decode_prefix.weight: copying a param with shape torch.Size([768, 64]) from checkpoint, the shape in current model is torch.Size([768, 768]).
	You may consider adding `ignore_mismatched_sizes=True` in the model `from_pretrained` method.

In [ ]:
from transformers import CLIPVisionModel, CLIPImageProcessor

cv = CLIPVisionModel.from_pretrained("openai/clip-vit-large-patch14")
cp = CLIPImageProcessor.from_pretrained("openai/clip-vit-large-patch14")

In [ ]:
from diffusion.data.builder import build_dataset, build_dataloader
from aspect_ratio_sampler import AspectRatioBatchSampler
from torch.utils.data import RandomSampler

data_config = {
    'type': 'FlexibleInternalDataMS',
    'roots': [
        '/mnt/bn/us-aigc-temp/henry/coco_2014/val/val2014/',
        # '/mnt/bn/aigc-us/zjl/laion-coco-aesthetic/data_max1024/',
        # '/mnt/bn/aigc-us/zjl/openimages/data/',
        # '/mnt/bn/aigc-us/zjl/sharegpt4v_processed_data/data/'
    ],
    'json_lst': [
        '/mnt/bn/us-aigc-temp/henry/test.json',
    ],
    'load_vae_feat': False,
    'load_t5_feat': False
}
dataset = build_dataset(
    data_config, resolution=512, aspect_ratio_type='ASPECT_RATIO_512',
    real_prompt_ratio=0.0, max_length=77,
)
batch_sampler = AspectRatioBatchSampler(sampler=RandomSampler(dataset), dataset=dataset,
                                    batch_size=4, aspect_ratios=dataset.aspect_ratio, drop_last=True,
                                    ratio_nums=dataset.ratio_nums, valid_num=0)
dataloader = build_dataloader(dataset, batch_sampler=batch_sampler, num_workers=10)

/home/tiger/.local/lib/python3.9/site-packages/bytedmetrics/__init__.py:10: UserWarning: bytedmetrics is renamed to bytedance.metrics, please using `bytedance.metrics` instead of `bytedmetrics`
  warnings.warn("bytedmetrics is renamed to bytedance.metrics, please using `bytedance.metrics` instead of `bytedmetrics`")
/home/tiger/.local/lib/python3.9/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(
2024-07-23 12:51:35,448 - PixArt - INFO - Constructing dataset FlexibleInternalDataMS...
2024-07-23 12:51:35,449 - PixArt - INFO - T5 max token length: 77
2024-07-23 12:51:35,450 - PixArt - INFO - ratio of real user prompt: 0.0
2024-07-23 12:51:35,496 

In [ ]:
iter_loader = iter(dataloader)

In [ ]:
batch = next(iter_loader)

In [ ]:
def encode_image(x, device='cpu', dtype=torch.float32):
    if x.min() < 0:
        assert x.min() >= -1.
        x = x * .5 + .5
    
    processed_list = cp(x, do_rescale=False)['pixel_values']
    return torch.tensor(np.stack(processed_list)).to(device).type(dtype)

In [ ]:
z = encode_image(batch[0])
z = cv(z).last_hidden_state

In [ ]:
text = [text_tokenizer.bos_token + txt + text_tokenizer.eos_token for txt in batch[1]]
out = text_tokenizer(text, return_tensors='pt', truncation=True, padding="longest")
print(out)
text_tokenizer.batch_decode(out['input_ids'])

{'input_ids': tensor([[50256,    32,   582, 10311,   257,  6769,   319,  1353,   286,   257,
          9053,  3526,    13, 50257, 50257],
        [50256, 15439,   319,  6729,  3526, 14284,   287,  1633,   351, 12269,
           287,   262,  4469,    13, 50257],
        [50256,    32,  2042,   290,  2330,  3797,   318,  5586,  2045,   503,
           257,  4324,    13, 50257, 50257],
        [50256,   464, 14580,   286,   257,  1323,   287,   257,  4038, 10162,
         50257, 50257, 50257, 50257, 50257]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]])}


['<|endoftext|>A man riding a wave on top of a surfboard.<|EOS|><|EOS|>',
 '<|endoftext|>Person on snowboard jumping in air with mountains in the background.<|EOS|>',
 '<|endoftext|>A black and white cat is sitting looking out a window.<|EOS|><|EOS|>',
 '<|endoftext|>The reflection of a bus in a vehicle mirror<|EOS|><|EOS|><|EOS|><|EOS|><|EOS|>']

In [ ]:
def pad_mask(orig_mask, prefix_len=77):
    extra_zeros = torch.zeros(size=(orig_mask.shape[0], 77), dtype=orig_mask.dtype, device=orig_mask.device)
    return torch.cat([extra_zeros, orig_mask], axis=1)

def pad_ids(input_ids, prefix_len=77):
    extra_zeros = torch.zeros(len(input_ids), prefix_len, dtype=torch.int64, device=input_ids.device)
    return torch.cat([extra_zeros, input_ids], axis=1)

In [ ]:
z[:, :77].shape

torch.Size([4, 77, 1024])

In [ ]:
out['input_ids']

tensor([[50256,    32,   582, 10311,   257,  6769,   319,  1353,   286,   257,
          9053,  3526,    13, 50257, 50257],
        [50256, 15439,   319,  6729,  3526, 14284,   287,  1633,   351, 12269,
           287,   262,  4469,    13, 50257],
        [50256,    32,  2042,   290,  2330,  3797,   318,  5586,  2045,   503,
           257,  4324,    13, 50257, 50257],
        [50256,   464, 14580,   286,   257,  1323,   287,   257,  4038, 10162,
         50257, 50257, 50257, 50257, 50257]])

In [ ]:
text_decoder.transformer.transformer.wte.weight.shape

torch.Size([50258, 768])

In [ ]:
mask = pad_mask(out['attention_mask'], prefix_len=z.shape[1])
# mask = out['attention_mask']
llm_out = text_decoder.forward(out['input_ids'], z[:, :77])#, attention_mask=mask)
llm_out.loss

tensor(14.1607, grad_fn=<NllLossBackward0>)

In [ ]:
def token_discrete_loss(logits, input_ids):
    loss_fct = torch.nn.CrossEntropyLoss(reduction="none")
    decoder_nll = loss_fct(logits.view(-1, logits.size(-1)), input_ids.view(-1)).view(
        input_ids.shape
    )
    decoder_nll = decoder_nll.mean(dim=-1)
    return decoder_nll

# token_discrete_loss(llm_out.logits[:, -7:], out['input_ids'])
token_discrete_loss(llm_out.logits, pad_ids(out['input_ids'])) - llm_out.loss

tensor([ 0.3160, -0.2841,  1.7209,  0.2218], grad_fn=<SubBackward0>)

In [ ]:
llm_out.logits.shape

torch.Size([4, 92, 50258])

In [ ]:
# from diffusers import UniDiffuserPipeline

# pipe = UniDiffuserPipeline.from_pretrained("thu-ml/unidiffuser-v1")

In [ ]:
# # pipe.text_decoder.encode_prefix = nn.Identity()
# # pipe.text_decoder.decode_prefix = nn.Identity()
# pipe.text_decoder.save_pretrained('/mnt/bn/us-aigc-temp/henry/unilatent_weights/gpt/')

In [ ]:
# pipe.text_tokenizer.save_pretrained('/mnt/bn/us-aigc-temp/henry/unilatent_weights/gpt_tokenizer/')